pip install paho-mqtt

!pip install pymysql

pip install --upgrade certifi

pip install pymysqlpool

pip install scikit-learn==1.3.1

In [ ]:
import paho.mqtt.client as mqtt
import pymysql
import json
from datetime import datetime, timedelta
import time
import threading
import numpy as np
import joblib

# 쓰레드를 정지시킬 이벤트
stop_event = threading.Event()

# 머신러닝 모델 불러오기
model_filename = "posture_model.pkl"
loaded_model = joblib.load(model_filename)

# MySQL 연결 설정
connection = pymysql.connect(
    host='project-db-campus.smhrd.com',
    port=3307,
    user='ttap',
    password='1234',
    database='ttap',
    cursorclass=pymysql.cursors.DictCursor
)

nowposture = None
nowlevel = None
dplevel = None
cplevel = None  
nowposture_temp = None
nowposture_temp_cnt = 0

# 사용자 설정 높이 조회
def check_height():
    print("check_height")
    global nowposture, nowlevel, dplevel, cplevel, nowposture_temp, nowposture_temp_cnt
    try:
        with connection.cursor() as cursor:
            sql = "SELECT cplevel, dplevel, nowposture, nowlevel FROM user_settings WHERE member_id = 'test1'"
            cursor.execute(sql)
            result = cursor.fetchone()
            if result:
                nowposture = result['nowposture']
                nowlevel = result['nowlevel']
                dplevel = result['dplevel']
                cplevel = result['cplevel']
                nowposture_temp = nowposture
                print(f"베개자세 : {nowposture} / 베개레벨 : {nowlevel} / DP레벨 : {dplevel} / CP레벨 : {cplevel} / 현재자세 : {nowposture_temp}")
    except pymysql.Error as e:
        print(f"Error checking height: {e}")

# 사용자 높이 변경
def change_height():
    print("change_height")
    global nowposture, nowlevel, dplevel, cplevel, nowposture_temp, nowposture_temp_cnt
    print(f"베개자세 : {nowposture} / 베개레벨 : {nowlevel} / DP레벨 : {dplevel} / CP레벨 : {cplevel}")
        
    air_check = ""
    air_level = 0

    if nowposture == "DP":
        if dplevel - nowlevel > 0:
            air_check = "in"
            air_level = dplevel - nowlevel
        elif dplevel - nowlevel < 0:
            air_check = "out"
            air_level = nowlevel - dplevel
        nowlevel = dplevel
    elif nowposture == "CP":
        if cplevel - nowlevel > 0:
            air_check = "in"
            air_level = cplevel - nowlevel
        elif cplevel - nowlevel < 0:
            air_check = "out"
            air_level = nowlevel - cplevel
        nowlevel = cplevel
        
    if air_level != 0:
        client.publish("Apill/height/return", json.dumps({"air_check": air_check, "air_level": air_level}))

        
# 시용자 현재 자세 변경
def change_nowposlevel():
    print("change_nowposlevel")
    global nowposture, nowlevel, dplevel, cplevel, nowposture_temp, nowposture_temp_cnt
    try:
        # 데이터베이스 업데이트
        with connection.cursor() as cursor:
            sql = "UPDATE user_settings SET nowposture = %s, nowlevel = %s WHERE member_id = 'test1'"
            cursor.execute(sql, (nowposture, nowlevel,))
            connection.commit()
            
        print(f"베개자세 : {nowposture}, 베개레벨 : {nowlevel} 업데이트")

    except (json.JSONDecodeError, KeyError, pymysql.Error) as e:
        print(f"Error changing level: {e}")
        
# 사용자 설정 높이 변경 
def change_cplevel(payload):
    print("change_cplevel")
    global nowposture, nowlevel, dplevel, cplevel, nowposture_temp, nowposture_temp_cnt
    try:
        # 여기에서 필요한 유효성 검사 등을 수행
        if json.loads(payload)['index'] == 'change':
            new_cplevel = int(json.loads(payload)['level'])
        elif json.loads(payload)['index'] == 'plus':
            new_cplevel = cplevel + int(json.loads(payload)['level'])
        elif json.loads(payload)['index'] == 'minus':
            new_cplevel = cplevel - int(json.loads(payload)['level'])
        
        if new_cplevel > 10:
            new_cplevel = 10
        elif new_cplevel < 1:
            new_cplevel = 1
            
        # 데이터베이스 업데이트
        print(f"{new_cplevel}")
        with connection.cursor() as cursor:
            sql = "UPDATE user_settings SET cplevel = %s WHERE member_id = 'test1'"
            cursor.execute(sql, (new_cplevel,))
            connection.commit()

        cplevel = new_cplevel
        change_height()
        print(f"CP 레벨 업데이트: {cplevel}")

    except (json.JSONDecodeError, KeyError, pymysql.Error) as e:
        print(f"Error changing CP level: {e}")

def change_dplevel(payload):
    print("change_dplevel")
    global nowposture, nowlevel, dplevel, cplevel, nowposture_temp, nowposture_temp_cnt
    try:
        # 여기에서 필요한 유효성 검사 등을 수행
        if json.loads(payload)['index'] == 'change':
            new_dplevel = int(json.loads(payload)['level'])
        elif json.loads(payload)['index'] == 'plus':
            new_dplevel = dplevel + int(json.loads(payload)['level'])
        elif json.loads(payload)['index'] == 'minus':
            new_dplevel = dplevel - int(json.loads(payload)['level'])
            
        if new_dplevel > 10:
            new_dplevel = 10
        elif new_dplevel < 1:
            new_dplevel = 1
            
        # 데이터베이스 업데이트
        with connection.cursor() as cursor:
            sql = "UPDATE user_settings SET dplevel = %s WHERE member_id = 'test1'"
            cursor.execute(sql, (new_dplevel,))
            connection.commit()

        dplevel = new_dplevel
        change_height()
        print(f"DP 레벨 업데이트: {dplevel}")

    except (json.JSONDecodeError, KeyError, pymysql.Error) as e:
        print(f"Error changing DP level: {e}")
        
# 자세 로그       
def record_posture_change(posture_type):
    try:
        global nowposture_temp, nowposture_temp_cnt, nowposture

        # 현재 시간 가져오기
        current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

        # 자세가 변경되었을 때 시작 시간 기록
        if nowposture_temp != posture_type:
            nowposture_temp = posture_type
            nowposture_temp_cnt = 0
            start_time = current_time
            # 여기에 DB에 시작 시간을 추가하는 코드를 작성
            with connection.cursor() as cursor:
                sql = "INSERT INTO posture_history (member_id, posture, start_time) VALUES (%s, %s, %s)"
                cursor.execute(sql, ('test1', posture_type, start_time))
                connection.commit()

        # 자세가 변경되었을 때 종료 시간 기록
        end_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

        # 일정 시간(예: 30초) 동안 센서 값이 들어오지 않으면 자세 변경 종료로 간주
        if nowposture_temp_cnt >= 30:  # 여기서는 3초로 설정
            # 여기에 DB에 종료 시간을 추가하는 코드를 작성
            with connection.cursor() as cursor:
                sql = "UPDATE posture_history SET end_time = %s WHERE member_id = 'test1' AND posture = %s AND end_time IS NULL"
                cursor.execute(sql, (end_time, nowposture_temp))
                connection.commit()
                
    except Exception as e:
        print(f"Error recording posture change: {e}")

def process_sensor_data(payload):
    global nowposture, nowlevel, dplevel, cplevel, nowposture_temp, nowposture_temp_cnt
    try:
        # 문자열을 쉼표로 분할하고 양쪽 공백 제거
        sensor_values = [value.strip() for value in payload.replace('"', '').split(',') if value.strip()]

        if len(sensor_values) == 26:
            # 모든 값을 실수로 변환
            sensor_values = list(map(float, sensor_values))

            # NumPy 배열로 변환하고 reshape
            input_data = np.array(sensor_values).reshape(1, -1)

            # 예측
            predicted_label = loaded_model.predict(input_data)[0]

            # 현재 시간 가져오기
            current_time = datetime.now().strftime("%H:%M")

            with connection.cursor() as cursor:
                # 정보를 데이터베이스에 추가
                sql = "INSERT INTO sleepposture (posture, time) VALUES (%s, %s)"
                cursor.execute(sql, (predicted_label, current_time))
                connection.commit()

            print(f"베개자세 : {nowposture} 판단하는자세 : {predicted_label[-2:]}")
            
            # 분류 결과 확인
            if predicted_label[-2:] == 'DP':
                if nowposture_temp != 'DP':
                    nowposture_temp = 'DP'
                    nowposture_temp_cnt = 0
                elif nowposture_temp == 'DP':
                    nowposture_temp_cnt += 1
                    if nowposture_temp_cnt == 5:
                        nowposture = 'DP'
                        change_height()
                        change_nowposlevel()
                        print("DP 높이 변경")
                        record_posture_change('DP')
                print(f"현재 자세:{nowposture_temp} 현재 자세 카운트:{nowposture_temp_cnt}")
                        
            elif predicted_label[-2:] == 'CP':
                if nowposture_temp != 'CP':
                    nowposture_temp = 'CP'
                    nowposture_temp_cnt = 0
                elif nowposture_temp == 'CP':
                    nowposture_temp_cnt += 1
                    if nowposture_temp_cnt == 5:
                        nowposture = 'CP'
                        change_height()
                        change_nowposlevel()
                        print("CP 높이 변경")
                        record_posture_change('CP')
                print(f"현재 자세:{nowposture_temp} 현재 자세 카운트:{nowposture_temp_cnt}")

            # 예측 결과를 MQTT 메시지로 전송 (실수로 변환하여 전송)
#             client.publish("Apill/sleepposture", str(predicted_label))

#             print(f"Posture recorded - Sensor Values: {sensor_values}, Predicted Label: {predicted_label}, Time: {current_time}")

    except Exception as e:
        print(f"Error processing sensor data: {e}")

# 알람 목록을 저장할 리스트
alarms = []
alarm_sendtime = ''

# 알람 목록 조회 함수
def get_alarms():
    print("get_alarms")
    global alarms
    try:
        with connection.cursor() as cursor:
            # 알람 목록을 데이터베이스에서 조회
            sql = "SELECT id, time, isOn, isSelected FROM alarms"
            cursor.execute(sql)
            alarms = cursor.fetchall()

        # datetime 객체를 문자열로 변환 또는 None인 경우 그대로 둠
        for alarm in alarms:
            if isinstance(alarm['time'], datetime):
                alarm['time'] = alarm['time'].strftime("%H:%M")

    except pymysql.Error as e:
        print(f"Error getting alarms: {e}")
    
    print(alarms)

# 주기적으로 현재 시간을 확인하고 알람이 발생해야 하는지 여부를 판단하는 함수
def check_and_trigger_alarm_periodically():
    global alarms
    global alarm_sendtime
    while not stop_event.is_set():
        try:
            current_time = datetime.now().strftime("%H:%M")  # 초 부분을 0으로 만듦
            if alarm_sendtime != current_time:
                for alarm in alarms:
                    alarm_time = alarm['time']
                    if alarm_time == current_time and alarm['isOn']:
                        print("알람시간체크")
                        print(f"Publishing triggered message to Apill/alarm/return")
                        client.publish("Apill/alarm/return", "triggered")
                        alarm_sendtime = alarm_time
            time.sleep(10)  # 1초 대신 60초로 수정
        except Exception as e:
            print(f"Error in check_and_trigger_alarm_periodically: {e}")
            
# 알람 추가 함수
def add_alarm(payload):
    print("add_alarm")
    try:
        alarm_data = json.loads(payload)
        with connection.cursor() as cursor:
            # 알람 정보를 데이터베이스에 추가
            sql = "INSERT INTO alarms (time, isOn, isSelected) VALUES (%s, %s, %s)"
            cursor.execute(sql, (alarm_data['time'], alarm_data['isOn'], alarm_data['isSelected']))
            connection.commit()
        get_alarms()
        print(f"알람 추가: {alarm_data}")
    except (json.JSONDecodeError, pymysql.Error) as e:
        print(f"Error adding alarm: {e}")

# 알람 수정 함수
def update_alarm(payload):
    print("update_alarm")
    try:
        alarm_data = json.loads(payload)
        with connection.cursor() as cursor:
            # 알람 정보를 데이터베이스에서 수정
            sql = "UPDATE alarms SET time=%s, isOn=%s, isSelected=%s WHERE id=%s"
            cursor.execute(sql, (alarm_data['time'], alarm_data['isOn'], alarm_data['isSelected'], alarm_data['id']))
            connection.commit()
        get_alarms()
        print(f"알람 업데이트: {alarm_data}")
    except (json.JSONDecodeError, pymysql.Error) as e:
        print(f"Error updating alarm: {e}")

# 알람 삭제 함수
def delete_alarm(payload):
    print("delete_alarm")
    try:
        alarm_id = json.loads(payload)['id']
        with connection.cursor() as cursor:
            # 알람 정보를 데이터베이스에서 삭제
            sql = "DELETE FROM alarms WHERE id=%s"
            cursor.execute(sql, (alarm_id,))
            connection.commit()
        get_alarms()
        print(f"알람 삭제: {alarm_id}")
    except (json.JSONDecodeError, pymysql.Error) as e:
        print(f"Error deleting alarm: {e}")        
        
# SQL 문을 실행하고 결과를 MQTT로 전송하는 함수
def execute_sql_query(sql_query):
    print("execute_sql_query")
    try:
        with connection.cursor() as cursor:
            # SQL 실행
            cursor.execute(sql_query)
            # 결과 가져오기
            result = cursor.fetchall()
            # 결과를 MQTT 토픽에 전송
            client.publish("Apill/sql/return", json.dumps(result, default=str))
        print(f"sql문 전송완료 {result}")
    except pymysql.Error as e:
        error_message = f"Error executing SQL: {e}"
        print(error_message)
        client.publish("Apill/sql/return", json.dumps(error_message, default=str))

# 회원가입
def insert_join(payload):
    try:
        # alarm_id = json.loads(payload)['id']
        member_data = json.loads(payload)
        
        with connection.cursor() as cursor:
            # SQL 실행
            sql = '''
                    INSERT INTO members (
                        member_id, member_pw, member_name, member_birth, 
                        member_weight, member_height, member_gender, member_age
                    ) VALUES (
                        %s, %s, %s, %s, %s, %s, %s, %s
                    )
                    '''
            
            cursor.execute(sql,(
                member_data['id'],
                member_data['password'],
                member_data['name'],
                member_data['birth'],
                member_data['weight'],
                member_data['height'],
                member_data['gender'],
                member_data['age']
            ))
            
            # 영향 받은 행의 수 확인
            affected_rows = cursor.rowcount
            
            # 결과를 MQTT 토픽에 전송
            json_result = json.dumps(affected_rows, default=str)
            print(f"SQL Query Result: {json_result}")
            client.publish("Apill/join/return", json_result)
    except pymysql.Error as e:
        error_message = f"Error executing SQL: {e}"
        print(error_message)
        client.publish("Apill/join/return", json.dumps(error_message, default=str))
        
# 회원가입 아이디 중복 검사
def join_id_check(payload):
    try:
        print(payload)
        user_id = json.loads(payload)['id']
        sql = 'select count(*) from members where id = %s'
        with connection.cursor() as cursor:
            # SQL 실행
            cursor.execute(sql,(user_id))
            # 결과 가져오기
            result = cursor.fetchall()
            # 결과를 MQTT 토픽에 전송
            json_result = json.dumps(result, default=str)
            print(f"SQL Query Result: {json_result}")
            client.publish("Apill/join/idcheck/return", json_result)
    except pymysql.Error as e:
        error_message = f"Error executing SQL: {e}"
        print(error_message)
        client.publish("Apill/join/idcheck/return", json.dumps(error_message, default=str))  
        
# 프로필 불러오기
def load_profile(payload):
    try:
        id_data = json.loads(payload)
        with connection.cursor() as cursor:
            # 알람 정보를 데이터베이스에 추가
            sql = "select * from members where id = %s"
            cursor.execute(sql, (id_data['id']))
            profile_data = cursor.fetchall()
            
            client.publish("Apill/user/profile/return", profile_data)
        print(f"id_profile: {id_data}")
    except (json.JSONDecodeError, pymysql.Error) as e:
        print(f"Error load_profile: {e}")
            
# MQTT 브로커 설정
broker_address = "172.30.1.19"
broker_port = 1883

# MQTT 클라이언트 생성
client = mqtt.Client()

# MQTT 연결 시 호출되는 콜백 함수
def on_connect(client, userdata, flags, rc):
    print(f"Connected with result code {rc}")
    
    # 토픽 구독
    # 센서값
    client.subscribe("Apill/RB/sensor")
#     client.subscribe("Apill/sleepposture")

    # 알람
    client.subscribe("Apill/alarm/add")
    client.subscribe("Apill/alarm/update")
    client.subscribe("Apill/alarm/delete")
    client.subscribe("Apill/alarm/get")
    
    # 높이변경
    client.subscribe("Apill/height")
    
    # sql문 처리
    client.subscribe("Apill/sql")
    
    # 회원가입
    client.subscribe('Apill/join')
    client.subscribe('Apill/join/idcheck')
    
    # 프로필
    client.subscribe('Apill/user/profile')

# MQTT 메시지 수신 시 호출되는 콜백 함수
def on_message(client, userdata, msg):
    print(f"받은 메세지: {msg.payload.decode()}")

    # 센서값 처리
    if msg.topic == "Apill/RB/sensor":
        process_sensor_data(msg.payload.decode())

    # 알람 목록 조회 메시지 처리
    elif msg.topic == "Apill/alarm/get":
        get_alarms()
    # 알람 추가 메시지 처리
    elif msg.topic == "Apill/alarm/add":
        add_alarm(msg.payload.decode())
    # 알람 수정 메시지 처리
    elif msg.topic == "Apill/alarm/update":
        update_alarm(msg.payload.decode())
    # 알람 삭제 메시지 처리
    elif msg.topic == "Apill/alarm/delete":
        delete_alarm(msg.payload.decode())
        
    # 높이
    elif msg.topic == "Apill/height":
        if nowposture == "DP":
            change_dplevel(msg.payload.decode())
        elif nowposture == "CP":
            change_cplevel(msg.payload.decode())
            
    # SQL 문을 받아서 실행하고 결과를 MQTT로 전송
    elif msg.topic == "Apill/sql":
        execute_sql_query(msg.payload.decode())
        
    # 회원가입
    elif msg.topic == "Apill/join":
        insert_join(msg.payload.decode())
    elif msg.topic == "Apill/join/idcheck":
        join_id_check(msg.payload.decode())
        

# MQTT 클라이언트에 콜백 함수 등록
client.on_connect = on_connect
client.on_message = on_message

# MQTT 브로커에 연결
client.connect(broker_address, broker_port, 60)

check_height()
change_height()
get_alarms()

# 시작
# 알람을 주기적으로 확인하는 쓰레드 시작
alarm_thread = threading.Thread(target=check_and_trigger_alarm_periodically)
alarm_thread.start()

# 브로커 서버 루프 시작
try:
    client.loop_forever()
except KeyboardInterrupt:
    print("KeyboardInterrupt: Stopping threads and disconnecting MQTT.")
    stop_event.set()  # 쓰레드 종료를 위해 이벤트 설정
    alarm_thread.join()  # 쓰레드 종료 대기
    client.disconnect()  # MQTT 연결 종료
except Exception as e:
    print(f"Unhandled exception: {e}")

print("Exiting program.")

check_height
베개자세 : CP / 베개레벨 : 7 / DP레벨 : 3 / CP레벨 : 9 / 현재자세 : CP
change_height
베개자세 : CP / 베개레벨 : 7 / DP레벨 : 3 / CP레벨 : 9
get_alarms
[{'id': 1, 'time': '09:53', 'isOn': 1, 'isSelected': 0}, {'id': 2, 'time': '19:17', 'isOn': 1, 'isSelected': 0}, {'id': 51, 'time': '19:15', 'isOn': 1, 'isSelected': 0}, {'id': 52, 'time': '19:18', 'isOn': 1, 'isSelected': 0}, {'id': 53, 'time': '19:28', 'isOn': 1, 'isSelected': 0}, {'id': 54, 'time': '09:43', 'isOn': 1, 'isSelected': 0}, {'id': 55, 'time': '09:56', 'isOn': 1, 'isSelected': 0}, {'id': 56, 'time': '10:19', 'isOn': 1, 'isSelected': 0}, {'id': 57, 'time': '12:13', 'isOn': 1, 'isSelected': 0}, {'id': 58, 'time': '15:23', 'isOn': 1, 'isSelected': 0}, {'id': 59, 'time': '15:19', 'isOn': 1, 'isSelected': 0}, {'id': 60, 'time': '15:19', 'isOn': 1, 'isSelected': 0}, {'id': 61, 'time': '15:19', 'isOn': 1, 'isSelected': 0}, {'id': 62, 'time': '15:26', 'isOn': 1, 'isSelected': 0}, {'id': 63, 'time': '15:20', 'isOn': 1, 'isSelected': 0}, {'id': 64,